# AI powered Highlight Clipper
Use the whisper API to get transcipts and then use ChatGPT for sentiment analysis to get if its interesting or not.

In [4]:
%pip install pytube
%pip install youtube-dl
%pip install openai
%pip install moviepy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import config

import pytube

import requests
import subprocess
import os
import shutil
import ast

import openai

from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.audio.io.AudioFileClip import AudioFileClip

import youtube_dl

In [3]:
openai.api_key = config.OAI_API_TOKEN

In [4]:
# YouTube video URL
vid_url = "https://youtu.be/9fvETktnaRw"
fps = 30

# how long the testing clips should be
testing_clips_time = 20

# How high the sentiment has to be for it to be in the video
sentiment_score = 0.6
max_token_count = 3000
custom_prompt = "entertaining a casual viewer of Youtube would find the sentence as part of a video"

# debug
delete_directories = False

In [8]:
# Get the audio stream
try:
    video_stream = pytube.YouTube(vid_url).streams.filter(only_audio=True).first()
except:
    video_stream = pytube.YouTube(vid_url).streams.filter(progressive=True).order_by('resolution').first()

# Check if audio stream is not None before attempting to download it
if video_stream is not None:
    # Download the audio stream
    audio_file = video_stream.download(filename="input.mp4").replace("\\","/")
else:
    print("No audio stream found.")


Break the video down into 25mb chunks

In [9]:
def split_video_by_seconds(input_file_path, chunk_duration):
    """
    Splits a video file into smaller chunks using ffmpeg.
    """
    # Get the duration of the input video
    duration_command = ["ffprobe", "-i", input_file_path, "-show_entries", "format=duration", "-v", "quiet", "-of", "csv=p=0"]
    duration_output = subprocess.check_output(duration_command, encoding="utf-8")
    input_duration = float(duration_output.strip())

    # Calculate the number of chunks and their start and end times
    chunk_start_times = range(0, int(input_duration), int(chunk_duration))
    chunk_end_times = [min(start_time + chunk_duration, input_duration) for start_time in chunk_start_times]

    # Split the video into chunks using ffmpeg
    for i, (start_time, end_time) in enumerate(zip(chunk_start_times, chunk_end_times)):
        output_file_path = f"downloads/input_{i}.mp4"
        split_command = ["ffmpeg", "-i", input_file_path, "-ss", str(start_time), "-to", str(end_time), "-c", "copy", output_file_path]
        subprocess.run(split_command, check=True)

newpath = os.getcwd() + "/downloads/"
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
input_file_path = 'input.mp4'
split_video_by_seconds(input_file_path, testing_clips_time)

Create metadata for the files <br/>
<sub>Start time and duration</sub>

In [10]:
newpath = os.getcwd() + "/metadata/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

start_time = 0.0

for i, filename in enumerate(os.listdir('downloads')):
    md_output_file_path = f"metadata/input_{i}.txt"
    with open(md_output_file_path, 'w') as output_file:
        clip = AudioFileClip('downloads/'+filename)
        duration = clip.duration
        clip.close()
        output_file.write(f"[{start_time},{duration}]")
        start_time += duration

Input the audio into Whisper and get transcription


In [11]:
newpath = os.getcwd() + "/sentences/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

url = 'https://api.openai.com/v1/audio/transcriptions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}
data = {
    'model':'whisper-1',
    'prompt':'Woah! I for a fact don\'t believe that... Okay, I know. I love those~'
}
for chunk_num, filename in enumerate(os.listdir('downloads')):
    with open("downloads/" + filename, "rb") as input_file:
        files = {
            'file':(filename,input_file)
        }
        transcription = requests.post(url=url, headers=headers, files=files, data=data)
        output_file_path = f"sentences/input_{chunk_num}.txt"
        with open(output_file_path, 'w', encoding="utf-8") as output_file:
            output_file.write(transcription.json().get('text'))

### ChatGPT Implementation

In [7]:
url = 'https://api.openai.com/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {config.OAI_API_TOKEN}'
}

Breakdown the text to the max_token_count

In [5]:
newpath = os.getcwd() + "/temporary/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

chunk_num = 0
for filename in os.listdir('sentences'):
    with open("sentences/" + filename, "r", encoding="utf-8") as chunk_file, \
         open("metadata/" + filename, "r", encoding="utf-8") as timestamp_file:
        chunk = chunk_file.read().strip() # read the entire chunk into memory
        timestamp = timestamp_file.read().strip() # read the entire timestamp into memory
        
        if not chunk: # skip empty chunks
            continue
        
        temporary_file_path = f"temporary/input_{chunk_num}.txt"
        if os.path.exists(temporary_file_path):
            with open(temporary_file_path, 'r', encoding="utf-8") as temporary:
                token_count = sum(len(line.strip().split()) for line in temporary)
                if token_count + len(chunk.split()) + len(timestamp.split()) + 6 > max_token_count:
                    chunk_num += 1
                    temporary_file_path = f"temporary/input_{chunk_num}.txt" # generate a new temporary file path
        
        with open(temporary_file_path, 'a', encoding="utf-8") as temporary:
            temporary.write(f"[{timestamp},\"{chunk}\"],") # write the chunk and timestamp as a JSON array
            
    chunk_num += 1

Use ChatGPT to break down the text and meta data into workable chunks

In [10]:
newpath = os.getcwd() + "/video_transcription/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

for chunk_num, filename in enumerate(os.listdir('temporary')):
    with open(f"temporary/input_{chunk_num}.txt","r", encoding="utf-8") as input_file:
        data = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": f"Using this list of [start_time, duration, video_transcription]. Respond with only a new list of chunks of the text you as an AI model find appropriately fits in speaking english by combinding list elements and then getting the new start_time by getting the first start_time of that chunk, then get the duration by adding the duration times. Get the new start and duration together to create the new list of [start_time, duration, chunk]. DO NOT give any answer outisde of the format. \n\"{input_file.read()}\""}] 
        }
        response = requests.post(url=url, headers=headers, json=data)
        created_chunks = response.json().get('choices')[0].get('message').get('content')
        with open(f"video_transcription/input_{chunk_num}.txt", "w", encoding="utf-8") as vidtrans:
            vidtrans.write(created_chunks)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'temporary'

Get sentiment from ChatGPT and create sentiment analysis file

In [ ]:
newpath = os.getcwd() + "/sentiment_analysis/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

for chunk_num, filename in enumerate(os.listdir('video_transcription')):
    with open(f"video_transcription/input_{chunk_num}.txt", "r", encoding="utf-8") as input_file:
        data = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": f"Using the list [start_time, duration, sentence]. For how '{custom_prompt}' in any definition to the best of your ability as an AI model the sentence is create a score as a float value to the best of your ability as an AI model. Only respond with a new list of [start_time, duration, sentence, float_score]. DO NOT give any answer outisde of the format. \n\"{input_file.read()}\""}] 
        }
        response = requests.post(url=url, headers=headers, json=data)
        sentiment = response.json().get('choices')[0].get('message').get('content')
        with open(f"sentiment_analysis/input_{chunk_num}.txt","w", encoding="utf-8") as sentanal: # hehe
            sentanal.write(sentiment)

Create Video from choosen segments

In [ ]:
# Get the audio stream
video_stream = pytube.YouTube(vid_url).streams.get_highest_resolution()
# Check if audio stream is not None before attempting to download it
if video_stream is not None:
    # Download the audio stream
    video_file = video_stream.download(filename="input_video.mp4").replace("\\","/")
else:
    print("No audio stream found.")

In [ ]:
newpath = os.getcwd() + "/videos/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

for chunk_num, filename in enumerate(os.listdir('sentiment_analysis')):
    with open(f"sentiment_analysis/input_{chunk_num}.txt","r", encoding="utf-8") as sentanal: # again hehe
        for i, sentlist in enumerate(ast.literal_eval(sentanal.read())):
            if(float(sentlist[3]) >= sentiment_score):
                output_file_path = f"videos/output_{i}.mp4"
                split_command = ["ffmpeg", "-framerate",fps,"-i", "input_video.mp4", "-ss", str(sentlist[0]), "-t", str(sentlist[1]), "-c", "copy", output_file_path]
                subprocess.run(split_command, check=True)

Combine the videos

In [ ]:
newpath = os.getcwd() + "/videos_avi/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

video_files_with_path = []

# List of video files to combine
for video_files in os.listdir("videos").sort(key=lambda x: int(re.sub(r'input_(\d+)\.mp4', r'\1', x)))
    path = "videos_avi/" + video_files.split(".")[0] + ".avi"
    commands = ['ffmpeg', '-i', "videos/" + video_files, '-c:v', 'copy', path]
    video_files_with_path.append(path)


# Create a list of commands to concatenate the videos
commands = ['ffmpeg', '-i', f'concat:{"|".join(video_files_with_path)}', '-c', 'copy', 'output.mp4']

# Run the ffmpeg command
subprocess.call(commands)

Delete directories

In [ ]:
if(delete_directories):
    newpath = ["/metadata/","/downloads/","/sentences/","/chunks/","/videos/","/video_transcription","/sentiment_analysis"]
    for i in newpath:
        if os.path.exists(os.getcwd() + i):
            shutil.rmtree(os.getcwd() + i)

    newfilepath = ["/input.mp4","/input_video.mp4"]
    for i in newfilepath:
        if os.path.exists(os.getcwd() + i):
            os.remove(os.getcwd() + i)